In [1]:
def gini_criterion(x: list, y: list, print_=False)-> int: 
    import numpy as np
    """Вычисление критерия Джини"""
    n_classes  = len(set(y)) # всего уникальных класов
    n_examples = len(x) # всего примеров
    
    def border_list(x: list) -> list:
        """Принимаем список значений, возвращаем индексы элементов, 
        после которых нужно попробовать провести границу"""
        border_list = []

        for idx, ex in enumerate(x):
            if idx != (n_examples - 1): # если индекс не последний
                try:
                    if y[idx+1] != y[idx]: # если следуюший элемент отличается
                        border_list.append(idx) # сохраняем в список
                    else:
                        pass
                except:
                    pass
            else:
                pass
        return border_list
    # список с массивами частот встречаемости классов (отсортированных по порядку)
    H_general_cls_freq = [np.where(np.array(y) == cls)[0].shape[0] \
                          for cls in sorted(list(set(y)))]
    # значение H(R) для всего списка
    H_general = sum([f/len(x)*(1-f/len(x)) for f in H_general_cls_freq])
    
    borders = border_list(x)
    
    res_dict = {}
    
    for b in borders:
        # разбиваем списки по индексу из списка границ
        x_left, y_left = x[:b+1], y[:b+1]
        x_right, y_right = x[b+1:], y[b+1:]
        #  значение H(Rleft) для левой части списка
        H_left_cls_freq = [np.where(np.array(y_left) == cls)[0].shape[0] \
                           for cls in sorted(list(set(y_left)))]
        H_left = sum([f/len(x_left)*(1-f/len(x_left)) for f in H_left_cls_freq])
        #  значение H(Rright) для правой части списка
        H_right_cls_freq = [np.where(np.array(y_right) == cls)[0].shape[0] \
                            for cls in sorted(list(set(y_right)))]
        H_right = sum([f/len(x_right)*(1-f/len(x_right)) for f in H_right_cls_freq])
        #  значение information gain по формуле
        IG = round(H_general - H_left*len(x_left)/len(x) - H_right*len(x_right)/len(x), 3)
        
        res_dict[b]=IG
                
#         print(IG)
    # индекс границы, для которой IG максимально
    res = max(res_dict, key=res_dict.get)
    if print_:
        print(f"Проводим границу после элемента с индексом {res}")
    
    return 
            
gini_criterion([1,2,3,4,5,6,7,8,9], [0,1,0,0,1,1,0,1,1], print_= True)

Проводим границу после элемента с индексом 3
